<a href="https://colab.research.google.com/github/ESLAMYASSER-1/Pytorch_learning_Daniel_Bourke/blob/main/Second.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import seaborn as sns